In [22]:
import pandas as pd
import sys
import os
import datetime
import numpy as np
import keras

from keras.models import Sequential
from keras.layers import InputLayer, Dense
import keras.optimizers as optimisers
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler, StandardScaler
sys.path.insert(0, os.path.abspath('../')) 

from lib.common import constants as k
from lib.eirgrid import data as eirgrid_data
from lib.common.marginal_emissions import compute_mef
from lib.common.data_window import DataWindow
import lib.common.gra as gra

In [6]:
eirgrid = eirgrid_data.system()
eirgrid = compute_mef(eirgrid).dropna()
eirgrid

,EffectiveTime,SysFrequency,Co2Emissions,Co2Intensity,SystemDemand,GenExp,InterNet,WindActual,MarginalEmissions
56,2023-12-28 09:30:00,49.98,676.0,146.0,4638.0,5298.0,-237.0,3798.0,-201.200000
57,2023-12-28 09:15:00,49.99,692.0,152.0,4555.0,5208.0,-213.0,3697.0,-1334.000000
58,2023-12-28 09:00:00,50.02,687.0,154.0,4473.0,5201.0,-288.0,3719.0,729.888889
59,2023-12-28 08:45:00,50.01,667.0,152.0,4385.0,5183.0,-770.0,3723.0,-67.756250
60,2023-12-28 08:30:00,49.95,682.0,159.0,4295.0,5023.0,-290.0,3511.0,159.000000
...,...,...,...,...,...,...,...,...,...
122476,2020-07-01 01:00:00,50.06,1506.0,414.0,3249.0,3542.0,-120.0,246.0,2834.419355
122477,2020-07-01 00:45:00,50.09,1600.0,435.0,3297.0,3573.0,-102.0,231.0,625.063694
122478,2020-07-01 00:30:00,50.07,1698.0,443.0,3376.0,3730.0,0.0,239.0,358.321168
122479,2020-07-01 00:15:00,50.02,1747.0,440.0,3453.0,3867.0,-235.0,243.0,12.333333


In [101]:
df = pd.read_csv(k.PROCESSED_DATA_DIR / 'features_target.csv', index_col=0)
target = df['MarginalEmissions']
features = df.drop(columns='MarginalEmissions')
features

,Fuel_Other_Renewable,SystemDemand,SysFrequency,ImbalanceSettlementPrice,Fuel_Gas,InterNet
56240,0.808739,0.343184,0.478261,0.095497,0.106146,0.606640
56241,0.804522,0.280015,0.304348,0.041720,0.089933,0.453011
56242,0.755447,0.221648,0.391304,0.054975,0.079715,0.552617
56243,0.750171,0.187662,0.521739,0.041642,0.069333,0.453011
56244,0.764685,0.155892,0.565217,0.044272,0.061194,0.453011
...,...,...,...,...,...,...
57711,0.842360,0.594016,0.521739,0.174926,0.151068,0.608329
57712,0.841677,0.522349,0.000000,0.427824,0.187820,0.200338
57713,0.829742,0.447728,0.956522,0.452772,0.099933,0.410242
57714,0.829486,0.386406,0.913043,0.154639,0.088831,0.572313


Setup train/test split

In [102]:
# Create feature columns
X = features
Y = target

# Train test split with 20% test size
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=1)

# Normalize the input data
# scaler  = StandardScaler()
# X_train = scaler.fit_transform(X_train)
# X_test  = scaler.transform(X_test)

# Setup validation set
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.25, random_state=1) # 0.25 x 0.8 = 0.2

Use Keras for MLP network

In [103]:
model = Sequential()

# Setup the input layer with n input features
model.add(InputLayer(input_shape=(X_train.shape[1], ), name="features"))

# Setup two hidden layers
# The first hidden layer will have 64 neurons (arbitrarily chosen)
model.add(Dense(64, activation="relu", name="hidden_1"))

# The second hidden layer will have 64 neurons
model.add(Dense(32, activation="relu", name="hidden_2")) 

# This will predict MEF and CO2 Intensity so two output layers
# Use a linear activation function for the output so we don't restirct or tamper the value
model.add(Dense(1, activation="linear", name="target"))

# Use adam optimiser to start
# Measure losses using MSE
model.compile(loss='mse', optimizer=optimisers.legacy.Adam())

# Summarise the model
model.summary()
(model.count_params() * 10), Y_train.size

Model: "sequential_27"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 hidden_1 (Dense)            (None, 64)                448       
                                                                 
 hidden_2 (Dense)            (None, 32)                2080      
                                                                 
 target (Dense)              (None, 1)                 33        
                                                                 
Total params: 2561 (10.00 KB)
Trainable params: 2561 (10.00 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


(25610, 881)

In [104]:
model.fit(X_train, Y_train,
          epochs=100,
          validation_data=(X_val, Y_val),
          callbacks=[keras.callbacks.EarlyStopping()])

Epoch 1/100
28/28 [==============================] - 0s 3ms/step - loss: 0.1572 - val_loss: 0.0065
Epoch 2/100
28/28 [==============================] - 0s 1ms/step - loss: 0.0098 - val_loss: 0.0050
Epoch 3/100
28/28 [==============================] - 0s 1ms/step - loss: 0.0053 - val_loss: 0.0033
Epoch 4/100
28/28 [==============================] - 0s 1ms/step - loss: 0.0044 - val_loss: 0.0026
Epoch 5/100
28/28 [==============================] - 0s 1ms/step - loss: 0.0036 - val_loss: 0.0019
Epoch 6/100
28/28 [==============================] - 0s 1ms/step - loss: 0.0031 - val_loss: 0.0016
Epoch 7/100
28/28 [==============================] - 0s 2ms/step - loss: 0.0028 - val_loss: 0.0014
Epoch 8/100
28/28 [==============================] - 0s 1ms/step - loss: 0.0024 - val_loss: 0.0011
Epoch 9/100
28/28 [==============================] - 0s 1ms/step - loss: 0.0023 - val_loss: 0.0015


In [105]:
model.evaluate(X_test, Y_test)

10/10 [==============================] - 0s 708us/step - loss: 0.0022


0.0021746729034930468